In [1]:
import multiprocessing
import pandas as pd
import numpy as np
import networkx as nx
import preprocessing as prep
import sys
import datetime

from multiprocessing import Pool
from bb_binary import FrameContainer, Repository, load_frame_container
from collections import namedtuple 
from pandas import DataFrame, Series

In [2]:
p = "/mnt/data/2015082215/"
c = .95
dist = 160
ilen = 3
cpus = 6

In [ ]:
pool = multiprocessing.Pool(cpus)

In [ ]:
repo = Repository(p)

In [ ]:
number_hours = 1*60*60 #number of hours in total in seconds
slice_len = 1*60   #number of seconds per slice

In [ ]:
m="08"
d="22"
h="15"
begin = "2015-{}-{}T{}:00:00Z".format(m,d,h) # %Y-%m-%dT%H:%M:%SZ
begin_ts = datetime.datetime.timestamp(datetime.datetime.strptime(begin, "%Y-%m-%dT%H:%M:%SZ"))

In [ ]:
parts = int(number_hours/slice_len)

In [ ]:
tasks = []

for i in list(range(parts)):
    b = begin_ts + (i * slice_len)
    e = (b-0.1) + (slice_len)
    #print("{}-{}".format(datetime.datetime.fromtimestamp(b),datetime.datetime.fromtimestamp(e)))
    tasks.append((p, b, e, c, dist, ilen))

In [ ]:
tasks[:2]

In [ ]:
def generate_network(path, b, e, confidence, distance, ilen):
    
    repo = Repository(path)
    xmax = 3000
    offset = 200
    
    Detection = namedtuple('Detection',
                           ['idx', 'xpos', 'ypos', 'radius', 'zRotation', 'decodedId', 'frame_idx', 'timestamp', 'cam_id', 'fc_id'])

    # one df per cam
    parts = np.empty(4, dtype=object)

    for i in list(range(4)):
         
        tpls = []
        myid = 0
        for frame, fc in repo.iter_frames(begin=b, end=e, cam=i):
            for d in frame.detectionsUnion.detectionsDP:
                d = Detection(d.idx, d.xpos, d.ypos, d.radius, d.zRotation, list(d.decodedId), myid, f.timestamp, fc.camId, fc.id)
                tpls.append(d)
            myid += 1
        
        df = DataFrame(tpls)
        df = prep.calcIds(df,confidence)
        parts[i] = df
        

    # cam 0 und cam1 nach rechts verschieben
    parts[0].xpos = parts[0].xpos + xmax + offset
    parts[1].xpos = parts[1].xpos + xmax + offset

    # Seiten zusammenfugen
    side0 = pd.concat([parts[3], parts[0]])
    side1 = pd.concat([parts[2], parts[1]])

    close1 = prep.get_close_bees_ckd(side0, distance)
    close2 = prep.get_close_bees_ckd(side1, distance)

    close = pd.concat([close1,close2])

    p = prep.bee_pairs_to_timeseries(close)

    return prep.extract_interactions(p,ilen)

In [ ]:
results = [pool.apply_async( generate_network, t ) for t in tasks[:2]]

In [ ]:
filename = "testframes"

In [ ]:
# Egebnisse zusammenbasteln
edges = []

for result in results:
    edges.append(result.get())

G = prep.create_graph2(pd.concat(edges))
nx.write_graphml(G, "{}_{}conf_{}dist_{}ilen".format(filename,str(c), str(dist), str(ilen)) + ".graphml")
print(nx.info(G))

# Ohne Multithreading